In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.ops import unary_union

# ----------------Predicting Total Vehicle Count in Georgia by 2025-------------------------------
# Step 1: Estimating Total Vehicle Count in Georgia by 2025
# Purpose: Predict the total number of vehicles in Georgia counties based on growth rates.

# Load the vehicle registration data
file_path = '/save/path/htaindex2019_data_counties_13/htaindex2019_data_counties_13.csv'
df = pd.read_csv(file_path)

# Add 'registered_car' column (households * autos per household)
df['registed_car'] = df['households'] * df['autos_per_hh_ami']

# Estimate vehicle growth rate based on U.S. and Georgia population growth
us_vehicle_growth_rate = 0.015  # 1.5% average U.S. growth rate 
us_population_growth = 7.4 / 100  # U.S. population growth (2010-2020) - from census report
georgia_population_growth = 10.6 / 100  # Georgia population growth (2010-2020) - from census report
georgia_vehicle_growth_rate = us_vehicle_growth_rate * (georgia_population_growth / us_population_growth)

# Predict vehicle count from 2020 to 2025
for year in range(2020, 2026):
    df[f'{year}_car'] = df['registed_car'] * (1 + georgia_vehicle_growth_rate) ** (year - 2019)


# Step 2: Classifying Counties into Urban, Suburban, and Rural Categories
# Purpose: Separate counties into three regions based on geographic urban areas.

# Load the Georgia Urban Area shapefile
tl_rd22_file_path = '/save/path/tl_rd22_us_uac20/tl_rd22_us_uac20.shp'
tl_rd22_gdf = gpd.read_file(tl_rd22_file_path)

# Filter for Georgia Urban Areas
georgia_urban_area_gdf = tl_rd22_gdf[
    tl_rd22_gdf['NAMELSAD20'].str.contains("GA Urban Area") &
    ~tl_rd22_gdf['NAMELSAD20'].str.contains("Eufaula, AL--GA Urban Area") |
    tl_rd22_gdf['NAMELSAD20'].isin(['Columbus, GA--AL Urban Area', 'Augusta-Richmond County, GA--SC Urban Area'])
]

# Load the county shapefile
county_shp_file_path = '/home/ojin/working_space/SIG/03. Data/00. Raw/16. census_county/tl_2023_us_county/tl_2023_us_county.shp'
county_gdf = gpd.read_file(county_shp_file_path)

# Identify counties that intersect with Atlanta Urban Area or other Urban Areas
atlanta_urban_area = georgia_urban_area_gdf[georgia_urban_area_gdf['NAMELSAD20'] == 'Atlanta, GA Urban Area']

# Assign region type (Urban, Suburban, Rural) based on spatial intersection
def assign_region(row, atlanta_urban_area, georgia_urban_area_gdf):
    if row.geometry.intersects(atlanta_urban_area.geometry.unary_union):
        return 'Urban'
    elif row.geometry.intersects(georgia_urban_area_gdf.geometry.unary_union):
        return 'Suburban'
    else:
        return 'Rural'

county_gdf['Region'] = county_gdf.apply(assign_region, atlanta_urban_area=atlanta_urban_area, georgia_urban_area_gdf=georgia_urban_area_gdf, axis=1)

# Step 3: Predicting 2025 Vehicle Count for Each County
# Purpose: Assign the predicted 2025 vehicle count to each county based on data.

csv_file_path = '/save/path/htaindex2019_data_counties_13/htaindex2019_data_counties_13_with_predicted_cars.csv'
df_csv = pd.read_csv(csv_file_path)
df_csv['county'] = df_csv['county'].str.replace('"', '')
df_csv.set_index('county', inplace=True)

# Merge the 2025 car data with county shapefile
county_gdf['GEOID'] = county_gdf['GEOID'].astype(str)
county_gdf['2025_car'] = county_gdf['GEOID'].map(df_csv['2025_car'])

# ----------------Estimating EVs Count in Georgia by 2025-------------------------------

# Step 4: Estimating the Number of EVs and Assigning EV Ports with OD Trip Data
# Purpose: Calculate the EV count and assign EV ports based on regional ratios. (From the reference: EERE report)

# Load OD trip data
trip_csv_file = '/save/path/OD_county.csv'
df_trip = pd.read_csv(trip_csv_file)
county_gdf = county_gdf.merge(df_trip[['county_NM', 'trip']], left_on='NAMELSAD', right_on='county_NM', how='left')

# Calculate total_trip and total_predicted_car
total_trip = county_gdf['trip'].sum()
total_predicted_car = county_gdf['2025_car'].sum()

# Calculate average_trip_per_car
average_trip_per_car = total_trip / total_predicted_car

# Add 'trip_based_car' column based on trip data
county_gdf['trip_based_car'] = county_gdf['trip'] / average_trip_per_car

# Weighted combination of 2025_car and trip_based_car (0.5:0.5)
weight_2025_car = 0.5
weight_trip_based = 0.5

def calculate_ev_car(row, w_a=weight_2025_car, w_b=weight_trip_based):
    # Weighted combination of 2025_car and trip_based_car
    weighted_value = (row['2025_car'] * w_a) + (row['trip_based_car'] * w_b)

    if row['Region'] == 'Rural':
        return round(weighted_value * 0.005)  # 0.5% for Rural
    elif row['Region'] == 'Urban':
        return round(weighted_value * 0.05)   # 5% for Urban
    elif row['Region'] == 'Suburban':
        return round(weighted_value * 0.015)  # 1.5% for Suburban
    else:
        return 0

county_gdf['EV_car'] = county_gdf.apply(calculate_ev_car, axis=1)

def calculate_ev_port(row):
    if row['Region'] == 'Urban':
        return row['EV_car'] * 1.5 / 1000  # Urban: 1.5port / 1,000 EV
    elif row['Region'] == 'Suburban':
        return row['EV_car'] * 2.2 / 1000  # Suburban: 2.2port / 1,000 EV
    elif row['Region'] == 'Rural':
        return row['EV_car'] * 3.1 / 1000  # Rural: 3.1port / 1,000 EV
    else:
        return 0

county_gdf['EV_port'] = county_gdf.apply(calculate_ev_port, axis=1)

# Save the result with adjusted EV ports to one of the key final files
output_file_county_with_ev_ports = '/save/path/DCFC_county_with_ev_ports.shp'
county_gdf.to_file(output_file_county_with_ev_ports)

# ----------------------------------------

# Step 6: Combining Urban and Suburban Regions
atlanta_file = '/save/path/Atlanta_merged/atlanta_merged.shp'
not_atlanta_file = '/save/path/final_ver7_Not_atlanta/final_ver7_suburban/DCFC_final_ver7_suburban.shp'

atlanta_gdf = gpd.read_file(atlanta_file)
not_atlanta_gdf = gpd.read_file(not_atlanta_file)
not_atlanta_gdf = not_atlanta_gdf.to_crs(atlanta_gdf.crs)

combined_gdf = pd.concat([atlanta_gdf, not_atlanta_gdf], ignore_index=True)

# Step 7: Distribute EV Ports by Area and Save
for county_idx, county_row in county_gdf.iterrows():
    intersecting_combined = combined_gdf[combined_gdf.intersects(county_row.geometry)]
    if not intersecting_combined.empty:
        total_area_in_county = intersecting_combined.intersection(county_row.geometry).area.sum()
        for idx, combined_row in intersecting_combined.iterrows():
            overlap_area = combined_row.geometry.intersection(county_row.geometry).area
            area_ratio = overlap_area / total_area_in_county if total_area_in_county != 0 else 0
            combined_gdf.at[idx, 'EV_port'] += round(county_row['EV_port'] * area_ratio)

# Save final urban/suburban file
output_file_final_urban_suburban = '/save/path/DCFC_final_ver7_urban_suburban.shp'
combined_gdf.to_file(output_file_final_urban_suburban)

# Step 8: Separate Urban and Suburban into Individual Files
urban_gdf = combined_gdf[combined_gdf['NAMELSAD20'].str.contains("Atlanta", na=False)]
suburban_gdf = combined_gdf[~combined_gdf['NAMELSAD20'].str.contains("Atlanta", na=False)]


# Save Urban and Suburban shapefiles
urban_output_file = '/save/path/Urban/DCFC_final_ver7_Urban.shp'
suburban_output_file = '/save/path/Suburban/DCFC_final_ver7_suburban.shp'

# Saving the urban and suburban shapefiles
urban_gdf.to_file(urban_output_file)
suburban_gdf.to_file(suburban_output_file)

print(f"Urban regions saved to {urban_output_file}")
print(f"Suburban regions saved to {suburban_output_file}")